In [10]:
from os import listdir
from pandas import read_csv
import pandas as pd
import numpy as np
import glob

In [23]:
# load a single file as a numpy array
def load_file(filepath):
   dataframe = read_csv(filepath, header=None, delim_whitespace=True)
   return dataframe.values

# load a group of files, such as x, y, z data for a given variable and  return them as a 3d numpy array
def load_group(filenames, prefix=''):
   loaded = list()
   for name in filenames:
      data = load_file(prefix + name)
      loaded.append(data)
      # stack group so that features are the 3rd dimension
   loaded = np.dstack(loaded)
   return loaded

# load a dataset type, such as train or test
def load_dataset(type, prefix=''):
   path = prefix + type + '/Inertial Signals/'

   filenames = list()
   # total acceleration
   filenames += ['total_acc_x_' + type + '.txt', 'total_acc_y_' + type + '.txt', 'total_acc_z_' + type + '.txt']
   # body acceleration
   filenames += ['body_acc_x_' + type + '.txt', 'body_acc_y_' + type + '.txt', 'body_acc_z_' + type + '.txt']
   # body gyroscope
   filenames += ['body_gyro_x_' + type + '.txt', 'body_gyro_y_' + type + '.txt', 'body_gyro_z_' + type + '.txt']

   # load input data
   X = load_group(filenames, path)
   # load output data
   y = load_file(prefix + type + '/y_'+type+'.txt')
   return X, y

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)


In [ ]:
# load all train data
trainX, trainy = load_dataset('train', 'UCI HAR Dataset/')
# load all test data
testX, testy = load_dataset('test', 'UCI HAR Dataset/')